## Search algorithms within Optuna

In this notebook, I will demo how to select the search algorithm with Optuna. We will compare the use of:

- Grid Search 
- Randomized search
- Tree-structured Parzen Estimators
- CMA-ES


We can select the search algorithm from the [optuna.study.create_study()](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.create_study.html#optuna.study.create_study) class.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

import optuna

In [2]:
# load dataset

X, y = load_breast_cancer(return_X_y=True, as_frame=True)
y = y.map({0:1, 1:0})

X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
# the target:
# percentage of benign (0) and malign tumors (1)

y.value_counts() / len(y)

target
0    0.627417
1    0.372583
Name: count, dtype: float64

In [4]:
# split dataset into a train and test set

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((398, 30), (171, 30))

## Define the objective function

This is the hyperparameter response space, the function we want to minimize.

In [5]:
# the objective function takes the hyperparameter space
# as input

def objective(trial):

    rf_n_estimators = trial.suggest_int("rf_n_estimators", 100, 1000)
    rf_criterion = trial.suggest_categorical("rf_criterion", ['gini', 'entropy'])
    rf_max_depth = trial.suggest_int("rf_max_depth", 1, 4)
    rf_min_samples_split = trial.suggest_float("rf_min_samples_split", 0.01, 1)
    
    model = RandomForestClassifier(
        n_estimators=rf_n_estimators,
        criterion=rf_criterion,
        max_depth=rf_max_depth,
        min_samples_split=rf_min_samples_split,
    )

    score = cross_val_score(model, X_train, y_train, cv=3)
    accuracy = score.mean()
    return accuracy

## Randomized Search

[RandomSampler()](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.RandomSampler.html)

In [6]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.RandomSampler(),
)


study.optimize(objective, n_trials=20)

[I 2024-09-20 16:41:02,347] A new study created in memory with name: no-name-e785a997-a9fd-4b7c-839d-146c31b68cf2
[I 2024-09-20 16:41:03,241] Trial 0 finished with value: 0.9422609554188502 and parameters: {'rf_n_estimators': 129, 'rf_criterion': 'entropy', 'rf_max_depth': 4, 'rf_min_samples_split': 0.015498666069839021}. Best is trial 0 with value: 0.9422609554188502.
[I 2024-09-20 16:41:05,044] Trial 1 finished with value: 0.9396597554492291 and parameters: {'rf_n_estimators': 215, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.33568182108535777}. Best is trial 0 with value: 0.9422609554188502.
[I 2024-09-20 16:41:05,894] Trial 2 finished with value: 0.9447482342219184 and parameters: {'rf_n_estimators': 101, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 0.02157539411677134}. Best is trial 2 with value: 0.9447482342219184.
[I 2024-09-20 16:41:10,880] Trial 3 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 671, 

In [7]:
study.best_params

{'rf_n_estimators': 101,
 'rf_criterion': 'gini',
 'rf_max_depth': 2,
 'rf_min_samples_split': 0.02157539411677134}

In [8]:
study.best_value

0.9447482342219184

In [9]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_rf_criterion,params_rf_max_depth,params_rf_min_samples_split,params_rf_n_estimators,state
0,0,0.942261,2024-09-20 16:41:02.353845,2024-09-20 16:41:03.240357,0 days 00:00:00.886512,entropy,4,0.015499,129,COMPLETE
1,1,0.939660,2024-09-20 16:41:03.241356,2024-09-20 16:41:05.043546,0 days 00:00:01.802190,entropy,2,0.335682,215,COMPLETE
2,2,0.944748,2024-09-20 16:41:05.045543,2024-09-20 16:41:05.893923,0 days 00:00:00.848380,gini,2,0.021575,101,COMPLETE
3,3,0.625636,2024-09-20 16:41:05.895922,2024-09-20 16:41:10.880339,0 days 00:00:04.984417,gini,1,0.678643,671,COMPLETE
4,4,0.922154,2024-09-20 16:41:10.881337,2024-09-20 16:41:15.901539,0 days 00:00:05.020202,entropy,4,0.504197,901,COMPLETE
5,5,0.937191,2024-09-20 16:41:15.902538,2024-09-20 16:41:17.588583,0 days 00:00:01.686045,gini,3,0.410570,290,COMPLETE
6,6,0.919648,2024-09-20 16:41:17.589584,2024-09-20 16:41:22.405112,0 days 00:00:04.815528,entropy,1,0.171496,729,COMPLETE
7,7,0.937191,2024-09-20 16:41:22.406111,2024-09-20 16:41:27.466208,0 days 00:00:05.060097,gini,4,0.427238,772,COMPLETE
8,8,0.937210,2024-09-20 16:41:27.467217,2024-09-20 16:41:33.384489,0 days 00:00:05.917272,entropy,3,0.155561,914,COMPLETE
9,9,0.917122,2024-09-20 16:41:33.385509,2024-09-20 16:41:36.825554,0 days 00:00:03.440045,entropy,1,0.598134,573,COMPLETE


## TPE

TPESampler is the default. You'll probably remember that for TPE, we first sample a few hyperparameter combinations at random, and then we start the sequential search, based on TPE with a certain kernel, and an acquisition function.

All these parameters are determined by default, but you can change them. More details in the [TPESampler()](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.TPESampler.html) documentation.

In [10]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(
        # the number of initial points to sample at random before doing the sequential search
        n_startup_trials=10
    ),
)

study.optimize(objective, n_trials=20)

[I 2024-09-20 16:42:05,959] A new study created in memory with name: no-name-8d5ce980-e21a-4536-97e9-8fc718c3efdf
[I 2024-09-20 16:42:11,001] Trial 0 finished with value: 0.9296157059314955 and parameters: {'rf_n_estimators': 808, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.3532166129886025}. Best is trial 0 with value: 0.9296157059314955.
[I 2024-09-20 16:42:15,209] Trial 1 finished with value: 0.9146350725298094 and parameters: {'rf_n_estimators': 648, 'rf_criterion': 'entropy', 'rf_max_depth': 1, 'rf_min_samples_split': 0.629082184954137}. Best is trial 0 with value: 0.9296157059314955.
[I 2024-09-20 16:42:18,601] Trial 2 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 292, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.8068326301179862}. Best is trial 0 with value: 0.9296157059314955.
[I 2024-09-20 16:42:23,397] Trial 3 finished with value: 0.9171223513328776 and parameters: {'rf_n_estimators': 622, 'r

In [11]:
study.best_params

{'rf_n_estimators': 956,
 'rf_criterion': 'entropy',
 'rf_max_depth': 4,
 'rf_min_samples_split': 0.012580167774078943}

In [12]:
study.best_value

0.9472734867471709

## CMA-ES

CmaEsSampler

In [13]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.CmaEsSampler(),
)

study.optimize(objective, n_trials=5)

[I 2024-09-20 16:43:34,872] A new study created in memory with name: no-name-bbf42233-4c1d-428f-ab72-536041e6f7ef
[I 2024-09-20 16:43:37,160] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 404, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.653168478527917}. Best is trial 0 with value: 0.6256360598465861.
[W 2024-09-20 16:43:37,180] The parameter 'rf_criterion' in trial#1 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2024-09-20 16:43:41,881] Trial 1 finished with value: 0.9296536796536796 and parameters: {'rf_n_estimators': 659, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.38

In [14]:
study.best_params

{'rf_n_estimators': 659,
 'rf_criterion': 'entropy',
 'rf_max_depth': 2,
 'rf_min_samples_split': 0.3883823738390945}

In [15]:
study.best_value

0.9296536796536796

## Grid Search

GridSampler()

We are probably not going to perform GridSearch with Optuna, but in case you wanted to, you need to add a variable with the space, with the exact values that you want to be tested.

In [16]:
search_space = {
    "rf_n_estimators": [100, 500, 1000],
    "rf_criterion": ['gini', 'entropy'],
    "rf_max_depth": [1, 2, 3],
    "rf_min_samples_split": [0.1, 1.0]
}


In [17]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.GridSampler(search_space),
)

study.optimize(objective)

[I 2024-09-20 16:43:53,151] A new study created in memory with name: no-name-cdaa301d-51cc-47f9-bf38-c0261bbf7a0e
[I 2024-09-20 16:43:56,211] Trial 0 finished with value: 0.9397167160325055 and parameters: {'rf_n_estimators': 500, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.1}. Best is trial 0 with value: 0.9397167160325055.
[I 2024-09-20 16:44:01,214] Trial 1 finished with value: 0.9221538695222905 and parameters: {'rf_n_estimators': 1000, 'rf_criterion': 'entropy', 'rf_max_depth': 1, 'rf_min_samples_split': 0.1}. Best is trial 0 with value: 0.9397167160325055.
[I 2024-09-20 16:44:03,396] Trial 2 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 500, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 1.0}. Best is trial 0 with value: 0.9397167160325055.
[I 2024-09-20 16:44:03,987] Trial 3 finished with value: 0.937210450368345 and parameters: {'rf_n_estimators': 100, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf

[I 2024-09-20 16:45:45,473] Trial 34 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 100, 'rf_criterion': 'gini', 'rf_max_depth': 1, 'rf_min_samples_split': 1.0}. Best is trial 23 with value: 0.942241968557758.
[I 2024-09-20 16:45:46,108] Trial 35 finished with value: 0.9171413381939697 and parameters: {'rf_n_estimators': 100, 'rf_criterion': 'gini', 'rf_max_depth': 1, 'rf_min_samples_split': 0.1}. Best is trial 23 with value: 0.942241968557758.


In [18]:
study.best_params

{'rf_n_estimators': 100,
 'rf_criterion': 'gini',
 'rf_max_depth': 3,
 'rf_min_samples_split': 0.1}

In [19]:
study.best_value

0.942241968557758